In [236]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'
# TRAJECTORY_NAME = 'vabadussild_novatel'
# TRAJECTORY_NAME = 'vabadusepst_novatel'
TRAJECTORY_NAME = 'oldtown_novatel'

# model_type = ModelType.NN
model_type = ModelType.NN_CLR
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [237]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()


Trial 35 Complete [00h 00m 02s]
val_loss: 0.07906805723905563

Best val_loss So Far: 0.0056214528158307076
Total elapsed time: 00h 00m 23s

Search: Running Trial #36

Value             |Best Value So Far |Hyperparameter
353               |890               |il_units
tanh              |relu              |il_activation
1                 |4                 |num_hidden_layers
0.3               |0.1               |huber_delta
205               |542               |hl_0_units
relu              |relu              |hl_0_activation
118               |938               |hl_1_units
tanh              |relu              |hl_1_activation
256               |32                |batch_size
39                |377               |hl_2_units
relu              |relu              |hl_2_activation
19                |680               |hl_3_units
tanh              |tanh              |hl_3_activation
753               |88                |hl_4_units
tanh              |relu              |hl_4_activation
3          

In [ ]:
processor.fit_trajectories()


In [ ]:
processor.save_trajectories()


In [ ]:
ape_fig = processor.create_ape_plot_all().show()


In [ ]:
processor.create_ape_plot().show()


In [ ]:
# fig_2d = processor.create_2d_plot_utm()
# fig_2d.show()


In [ ]:
# fig_slam = processor.create_2d_plot_slam()
# fig_slam.show()


In [ ]:
# fig_map = processor.create_map_plot()
# fig_map.show()
